In [1]:
# CNN ReLU
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,TensorDataset
import tqdm
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import torch.nn.functional as F

In [2]:
# 훈련용데이터
fashion_mnist_train = FashionMNIST('./FashionMNIST',train=True, download=True,transform=transforms.ToTensor() )
fashion_mnist_test = FashionMNIST('./FashionMNIST',train=False, download=True,transform=transforms.ToTensor() )

100%|██████████| 26421880/26421880 [00:01<00:00, 15952224.68it/s]


Extracting ./FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 277297.17it/s]


Extracting ./FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5039146.00it/s]


Extracting ./FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 9470296.93it/s]

Extracting ./FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/FashionMNIST/raw



In [3]:
# 배치 크기를 128
batch_size = 128
train_loader = DataLoader(fashion_mnist_train,batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test,batch_size=batch_size, shuffle=False)

In [4]:
for data,label in train_loader:
  print(data.shape)
  break

torch.Size([128, 1, 28, 28])


In [5]:
# torch.Size([128, 1, 28, 28])
# (batch,channel,height,width)  --> (batch, channelxheightxwidth)


In [6]:
# 2층 합성곱, 2층 MLP을 연결한 CNN
conv_net = nn.Sequential(
    nn.Conv2d(1,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),

    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25), # torch.Size([1, 64, 4, 4])
    nn.Flatten()  # (1 ,1024)
)

In [7]:
# 합성곱에 의해 최종적으로 이미지 크기가 어떤지를 더미데이터를 넣어서 확인
test_input = torch.ones(1, 1, 28, 28)
conv_out_put = conv_net(test_input).size()[-1]

In [8]:
# 2층 MLP
mlp = nn.Sequential(
    nn.Linear(conv_out_put,200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200,10)
)

In [9]:
# 최종 cnn
model = nn.Sequential(
    conv_net,mlp
)

In [10]:
model

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): Flatten(start_dim=1, end_dim=-1)
  )
  (1): Sequential(
    (0): Linear(in_features=1024, out_features=200, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
import torch.optim as optim

In [12]:
# 훈련
model.to(device)
op = optim.SGD(model.parameters(), lr = 1e-2, momentum=0.5)

In [13]:
# 학습함수
from tqdm import tqdm
for epoch in range(10):
  model.train()
  for idx , (data, label) in tqdm(enumerate(train_loader)):
    data, label = data.to(device), label.to(device)
    op.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, label)
    loss.backward()
    op.step()
    if idx % 100 == 0:
      print(f"train epoch:{epoch} loss : {loss.item()}")

5it [00:08,  1.24s/it]

train epoch:0 loss : 2.5147721767425537


105it [00:10, 40.44it/s]

train epoch:0 loss : 0.6033198833465576


210it [00:12, 53.40it/s]

...

308it [00:06, 55.39it/s]

train epoch:9 loss : 0.1639772355556488


410it [00:08, 54.78it/s]

train epoch:9 loss : 0.2013736069202423


469it [00:09, 49.98it/s]


In [14]:
# 평가
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
  for idx , (data, label) in tqdm(enumerate(test_loader)):
    data, label = data.to(device), label.to(device)
    
    output = model(data)      
    # 배치오차 합산
    test_loss += F.cross_entropy(output, label,reduction='sum').item()
    # 가장 높은 값을 가진 인덱스가 예측값
    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(label.view_as(pred)).sum().item()
test_loss /= len(test_loader.dataset)
test_accuracy = 100*correct / len(test_loader.dataset)
print(test_loss, test_accuracy)

79it [00:01, 58.13it/s]

0.270410431265831 90.15


In [17]:
# 전이학습
# 층이 매우깊고. 파라메터가 방대한 모델..
import gdown
gdown.download('https://github.com/leekyuyoung20221226/python/blob/main/data/taco_and_burrito.tar.gz?raw=true',
               './taco_and_burrito.tar')


Downloading...
From: https://github.com/leekyuyoung20221226/python/blob/main/data/taco_and_burrito.tar.gz?raw=true
To: /content/taco_and_burrito.tar
100%|██████████| 15.3M/15.3M [00:00<00:00, 132MB/s]


'./taco_and_burrito.tar'

In [ ]:
!tar -zxvf taco_and_burrito.tar

In [20]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [22]:
# ImageFolder를 이용해서 DataSet을 작성
train_imgs = ImageFolder('./taco_and_burrito/train/',transform=transforms.Compose([
    transforms.RandomCrop(224),
    transforms.ToTensor()
]))
test_imgs = ImageFolder('./taco_and_burrito/test/',transform=transforms.Compose([
    transforms.RandomCrop(224),
    transforms.ToTensor()
]))
train_loader = DataLoader(train_imgs, batch_size=32,shuffle=True)
test_loader = DataLoader(test_imgs, batch_size=32,shuffle=False)

In [23]:
train_imgs.classes

['burrito', 'taco']

In [24]:
train_imgs.class_to_idx

{'burrito': 0, 'taco': 1}

In [25]:
from torchvision import models

In [26]:
# resnet18 모델을 가져온다.
net = models.resnet18(pretrained=True)  # 학습된 가중치를 사용
# 가중치가 업데이트되어서는 안됨.
# 모든 파라메터는 미분 대상에서 제외
for p in net.parameters():
  p.requires_grad=False

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 174MB/s]


In [31]:
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim,2)

In [60]:
import numpy as np
np.infty

inf

In [77]:
import tqdm
def train_net(net, train_loader,  only_fc = True, op = optim.Adam,
              loss_fn = nn.CrossEntropyLoss(), n_iter=10,device='cpu'):
  train_losses = []
  tain_acc = []
  val_acc = []
  best_model = net
  best_acc = 1e-10
  if only_fc: #마지막 선형계층의 파라메터만 전달
    optimizer = op(net.fc.parameters())
  else:
    optimizer = op(net.parameters())
  for epoch in range(n_iter):
    running_loss = 0.0
    # 훈련모드로 변경
    net.train()
    acc = 0
    for idx, (data, label) in tqdm.tqdm(enumerate(train_loader)):
      data = data.to(device)
      label = label.to(device)

      pred = net(data)
      loss = loss_fn(pred,label)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()  
      _, y_pred =  pred.max(1)
      acc += (label == y_pred).float().sum().item()

    acc_ = acc / len(train_loader.dataset)
    print(f"train accuracy : {acc_}")

    train_losses.append(running_loss/idx)
    if best_acc < acc_:
      best_acc = acc_
      best_model = net
      print("chnaged models")
    # print(f"epoch:{epoch} train_loss:{train_losses[-1]}")
  return best_model

In [67]:
def eval_net(net, data_loader, device='cpu'):
  net.eval()
  acc = 0 
  for idx,(data,label) in enumerate(data_loader):
    data = data.to(device)
    label = label.to(device)
    with torch.no_grad():
      _, pred = net(data).max(1)
    acc += (pred == label).float().sum().item()  
  print(f"test accuracy : {acc / len(test_loader.dataset)}")

In [78]:
# 훈련
net.to(device)
best_net = train_net(net,train_loader,test_loader,n_iter=10, device=device)

23it [00:02,  9.04it/s]


train accuracy : 0.9199438202247191
chnaged models


23it [00:02,  9.02it/s]


train accuracy : 0.9283707865168539
chnaged models


23it [00:03,  7.44it/s]


...

train accuracy : 0.9058988764044944


23it [00:02,  9.13it/s]


train accuracy : 0.8848314606741573


23it [00:02,  9.26it/s]

train accuracy : 0.9129213483146067


In [79]:
eval_net(net,test_loader,device=device), eval_net(best_net,test_loader,device=device)

test accuracy : 0.85
test accuracy : 0.8166666666666667


(None, None)